<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_V20_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2024-05-20 15:02:20


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','EQUITAS', 'SCHAEFFLER ', 'ADANITRANS', 'UJJIVAN']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
#stock_ids

In [4]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()

In [5]:
import yfinance as yfin
def get_stock_v20(stock_symbol):
    #print(stock_symbol)
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Candle'] = 'R'
    stock_df.loc[stock_df['Close'] > stock_df['Open'], 'Candle'] = 'G'
    stock_df['Prev_Candle'] = stock_df['Candle'].shift(1)
    stock_df['Next_Candle'] = stock_df['Candle'].shift(-1)
    stock_df['Signal'] = 'NA'
    stock_df.loc[(stock_df['Candle'] != stock_df['Prev_Candle']) & (stock_df['Candle'] == 'G'), 'Signal'] = 'Start'
    stock_df.loc[(stock_df['Candle'] != stock_df['Next_Candle']) & (stock_df['Next_Candle'] == 'R') & (stock_df['Signal'] == 'NA'), 'Signal'] = 'End'
    stock_df.loc[(stock_df['Candle'] != stock_df['Next_Candle']) & (stock_df['Next_Candle'] == 'R') & (stock_df['Signal'] == 'Start'), 'Signal'] = 'Start/End'
    stock_df = stock_df[stock_df['Signal'] != 'NA']
    stock_df['Prev_Low'] = stock_df['Low'].shift(1)
    stock_df = stock_df[stock_df['Signal'] != 'Start']
    stock_df.loc[(stock_df['Signal'] == 'End'), 'Low'] = stock_df['Prev_Low']
    stock_df['Up%'] = round((stock_df['High'] - stock_df['Low'])*100/stock_df['Low'],2)
    stock_df = stock_df[stock_df['Up%'] > 20]
    stock_df['Current'] = yfin.Ticker(stock_symbol).history(period='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]['Close'].values[0]
    stock_df = stock_df[stock_df['Current'] < stock_df['Low']]
    stock_df['Stock'] = stock_symbol
    return stock_df

def get_v20():
    df_v20 = pd.DataFrame()
    for stock_id in stock_ids:
        #print(stock_id)
        df_tmp = get_stock_v20(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        #df_v20 = df_v20.append(df_tmp)
        df_v20 = pd.concat([df_v20, df_tmp])

    return df_v20.sort_index()

In [6]:
cols = ['Stock', 'Up%', '200_SMA', 'Close' ,'Criteria']
df_v20_prospects = get_v20()[cols]

In [7]:
df_v20_prospects_200SMA = df_v20_prospects[(df_v20_prospects['Close'] <= df_v20_prospects['200_SMA'])]

In [8]:
df_v20_prospects_V40N = df_v20_prospects[(df_v20_prospects['Criteria'] == 'AV40N') | (df_v20_prospects['Criteria'] == 'AV40')]

In [9]:
df_final_v20 = pd.concat([df_v20_prospects_V40N.reset_index(), df_v20_prospects_200SMA.reset_index()])
df_final_v20.set_index("Date", inplace = True)
df_final_v20.sort_index()

,Stock,Up%,200_SMA,Close,Criteria
Date,,,,,
2021-05-28 00:00:00+05:30,LUXIND.NS,61.36,2348.0,3233.427734,AV40N
2021-10-27 00:00:00+05:30,TTKPRESTIG.NS,23.52,875.0,1097.593140,AV40N
2021-11-18 00:00:00+05:30,SFL.NS,20.50,1180.0,1593.900024,AV40N
2021-12-20 00:00:00+05:30,TASTYBITE.NS,23.05,16718.0,14266.582031,NaN
2022-01-10 00:00:00+05:30,VALIANTORG.NS,23.21,1355.0,1277.878418,AVR
2022-01-12 00:00:00+05:30,SFL.NS,21.16,1281.0,1882.724976,AV40N
2022-01-27 00:00:00+05:30,LUXIND.NS,25.10,3674.0,2787.197266,AV40N
2022-01-27 00:00:00+05:30,LUXIND.NS,25.10,3674.0,2787.197266,AV40N
2022-02-03 00:00:00+05:30,TEAMLEASE.NS,20.78,4094.0,4627.649902,AV40N


In [10]:
df_v20_prospects['Criteria'].value_counts()

Criteria
AVR      21
AV40N    14
AAR      11
AVSP      9
EXP50     1
Name: count, dtype: int64